https://huggingface.co/docs/transformers/main/en/model_doc/llama

In [1]:
from transformers import LlamaForCausalLM, AutoTokenizer
import datasets
import os
import torch
import pandas as pd
import json
import numpy as np
torch.cuda.is_available()

True

In [2]:
path = '/mnt/data/zoo/llama2/llama2-7b-hf/'
tokenizer = AutoTokenizer.from_pretrained(path)
model = LlamaForCausalLM.from_pretrained(path, torch_dtype=torch.float16).cuda()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
input_ids = tokenizer(
    '',
    return_tensors="pt"
).input_ids  # Batch size 1

In [4]:
outputs = model.generate(input_ids.cuda(), max_new_tokens=100)
tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
# tokenizer.decode(outputs[0], skip_special_tokens=True)

/mnt/data/sonia/miniconda3/envs/llama/lib/python3.11/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


'Tags: c#, asp.net, asp.net-mvc, asp.net-mvc-4, asp.net-mvc-5\n\nQuestion: How to get the value of a checkbox in a view in ASP.NET MVC\n\nI have a view with a checkbox and a button.\n\n\\begin{code}\n<input type="checkbox" name="IsActive" value="true" />\n<input type="submit" value="Save" />\n\\'

In [5]:
outputs

tensor([[    1,   917, 29901,   274,  6552,  7051, 29889,  1212, 29892,  7051,
         29889,  1212, 29899, 12943, 29892,  7051, 29889,  1212, 29899, 12943,
         29899, 29946, 29892,  7051, 29889,  1212, 29899, 12943, 29899, 29945,
            13,    13, 16492, 29901,  1128,   304,   679,   278,   995,   310,
           263, 12527,   297,   263,  1776,   297, 12738, 29889,  6006, 13348,
            13,    13, 29902,   505,   263,  1776,   411,   263, 12527,   322,
           263,  2826, 29889,    13,    13, 29905,   463, 29912,   401, 29913,
            13, 29966,  2080,  1134,   543, 12348, 29908,  1024,   543,  3624,
          9966, 29908,   995,   543,  3009, 29908,  2900,    13, 29966,  2080,
          1134,   543,  7892, 29908,   995,   543, 11371, 29908,  2900,    13,
         29905]], device='cuda:0')

## Dataset

In [2]:
datapath = '/mnt/data/sonia/honeygan/data/processed/data.csv'
df = pd.read_csv(datapath)
df.head()

,os,ip_str,port,module,cpe,cpe_count,category,os_generic,single_cpe
0,Ubuntu,43.205.13.243,21,ftp,NaN,0,file_sharing,ubuntu,NaN
1,Ubuntu,43.205.13.243,22,ssh,['cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0...,1,remote_access,ubuntu,cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5
2,Ubuntu,43.205.13.243,80,http,['cpe:/a:igor_sysoev:nginx'],1,webserver,ubuntu,cpe:/a:igor_sysoev:nginx
3,Ubuntu,43.205.13.243,443,https,['cpe:/a:igor_sysoev:nginx'],1,webserver,ubuntu,cpe:/a:igor_sysoev:nginx
4,Windows (Build 6.3.9600),206.233.189.205,80,http,['cpe:/a:igor_sysoev:nginx'],1,webserver,windows,cpe:/a:igor_sysoev:nginx


In [5]:
zs = np.random.randint(low=0, high=9999, size=df.shape[0]) #latent variable

In [7]:
parsed=json.loads(df.to_json(orient='records'))
data_list=[{'input':str(z), 'output':str(r)} for z, r in zip(zs, parsed)]
data = datasets.Dataset.from_list(data_list)
data[:5]

{'input': ['9262', '1897', '4645', '2088', '3459'],
 'output': ["{'os': 'Ubuntu', 'ip_str': '43.205.13.243', 'port': 21, 'module': 'ftp', 'cpe': None, 'cpe_count': 0, 'category': 'file_sharing', 'os_generic': 'ubuntu', 'single_cpe': None}",
  '{\'os\': \'Ubuntu\', \'ip_str\': \'43.205.13.243\', \'port\': 22, \'module\': \'ssh\', \'cpe\': "[\'cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5\']", \'cpe_count\': 1, \'category\': \'remote_access\', \'os_generic\': \'ubuntu\', \'single_cpe\': \'cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5\'}',
  '{\'os\': \'Ubuntu\', \'ip_str\': \'43.205.13.243\', \'port\': 80, \'module\': \'http\', \'cpe\': "[\'cpe:/a:igor_sysoev:nginx\']", \'cpe_count\': 1, \'category\': \'webserver\', \'os_generic\': \'ubuntu\', \'single_cpe\': \'cpe:/a:igor_sysoev:nginx\'}',
  '{\'os\': \'Ubuntu\', \'ip_str\': \'43.205.13.243\', \'port\': 443, \'module\': \'https\', \'cpe\': "[\'cpe:/a:igor_sysoev:nginx\']", \'cpe_count\': 1, \'category\': \'webserver\', \'os_generic\':

In [8]:
data.save_to_disk('/mnt/data/sonia/honeygan/llama_format_jul25.dat')

Saving the dataset (0/1 shards):   0%|          | 0/1043945 [00:00<?, ? examples/s]

In [24]:
reloaded_dataset = datasets.load_from_disk('/mnt/data/sonia/honeygan/llama_format_jul25.dat')
reloaded_dataset